In [1]:
from pathlib import Path
import os
import torch
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import datetime



# Téléchargement des données

from datamaestro import prepare_dataset
ds = prepare_dataset("com.lecun.mnist");
train_images, train_labels = ds.train.images.data(), ds.train.labels.data()
test_images, test_labels =  ds.test.images.data(), ds.test.labels.data()

# Tensorboard : rappel, lancer dans une console tensorboard --logdir runs
writer = SummaryWriter("runs/runs"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

# Pour visualiser
# Les images doivent etre en format Channel (3) x Hauteur x Largeur
images = torch.tensor(train_images[0:8]).unsqueeze(1).repeat(1,3,1,1).double()/255.
# Permet de fabriquer une grille d'images
images = make_grid(images)
# Affichage avec tensorboard
writer.add_image(f'samples', images, 0)


savepath = Path("model.pch")


class MonDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        return image, label
    
    def __len__(self):
        return len(self.images)
    





In [9]:
print(train_images.shape)
print(test_images.shape)


(60000, 28, 28)
(10000, 28, 28)


In [13]:
import torch.optim as optim

In [15]:
num_epochs = 10
batch_sizes = [32, 128, 256]
log_interval = 100

input_size = 28 * 28  
hidden_size = 128  
output_size = 10  


for batch_size in batch_sizes:
    print("\nsize : ", batch_size)
    train_loader = DataLoader(MonDataset(train_images, train_labels), batch_size=batch_size, shuffle=True)
    
    model = nn.Sequential(
        nn.Linear(input_size, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size, output_size)
    )

    criterion = nn.CrossEntropyLoss()  
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        for batch_idx, (images, labels) in enumerate(train_loader):
            images = images.reshape(-1, 28 * 28)/255.
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, loss.item()))

Train Epoch: 0 	Loss: 0.162738
Train Epoch: 1 	Loss: 0.068157
Train Epoch: 2 	Loss: 0.161976
Train Epoch: 3 	Loss: 0.036995
Train Epoch: 4 	Loss: 0.046881
Train Epoch: 5 	Loss: 0.004175
Train Epoch: 6 	Loss: 0.013480
Train Epoch: 7 	Loss: 0.001900
Train Epoch: 8 	Loss: 0.088509
Train Epoch: 9 	Loss: 0.001741
Train Epoch: 0 	Loss: 0.207830
Train Epoch: 1 	Loss: 0.187557
Train Epoch: 2 	Loss: 0.122038
Train Epoch: 3 	Loss: 0.154844
Train Epoch: 4 	Loss: 0.146669
Train Epoch: 5 	Loss: 0.054450
Train Epoch: 6 	Loss: 0.091944
Train Epoch: 7 	Loss: 0.010414
Train Epoch: 8 	Loss: 0.020707
Train Epoch: 9 	Loss: 0.026456
Train Epoch: 0 	Loss: 0.226050
Train Epoch: 1 	Loss: 0.245057
Train Epoch: 2 	Loss: 0.155329
Train Epoch: 3 	Loss: 0.144343
Train Epoch: 4 	Loss: 0.189168
Train Epoch: 5 	Loss: 0.068299
Train Epoch: 6 	Loss: 0.084418
Train Epoch: 7 	Loss: 0.045084
Train Epoch: 8 	Loss: 0.107420
Train Epoch: 9 	Loss: 0.044527


In [17]:

class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded



In [18]:
input_dim = 784  
encoding_dim = 32  



In [19]:
input_dim = 784  
encoding_dim = 32 


criterion = nn.MSELoss()
n_hidden = 20

lr = 0.01
batch_size = 100
iterations = 10

train_loader = DataLoader(MonDataset(train_images, train_labels), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(MonDataset(test_images, test_labels), batch_size=batch_size, shuffle=True)


device = torch.device( 'cuda ' if torch.cuda.is_available() else 'cpu' )
print(f"running on {device}")
class State :
    def __init__(self, model, optim) :
        self.model = model
        self.optim = optim
        self.epoch, self.iteration = 0,0

if savepath.is_file():
    with savepath.open("rb") as fp:
        state = torch.load(fp)
else:
    model = Autoencoder(input_dim, encoding_dim)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    state = State(model, optimizer)
for epoch in range(iterations):
    for x,y in train_loader:
        x = x.reshape(-1, 28 * 28)/255.
        state.optim.zero_grad()
        x = x.to(device)
        outputs = state.model(x)
        loss = criterion(outputs, x)
        loss.backward()
        state.optim.step()
        state.iteration += 1
    with savepath.open("wb") as fp:
        state.epoch += 1
        torch.save(state, fp)
    with torch.no_grad():
        for x, _ in test_loader:
            x = x.reshape(-1, 28 * 28)/255.
            x = x.to(device)
            outputs = model(x)
            loss = criterion(outputs, x)




running on cpu
